In [1]:
import pandas as pd
data_directory = '../data/'

In [2]:
time_series_metadata = pd.DataFrame(columns=['Units', 'Source'])
time_series_metadata.index.name='Name'

In [3]:
df = pd.read_csv(data_directory+'original/Farmer_Lafond_Data.csv', index_col=0)

for col in df.columns:
    time_series_metadata.ix[col, 'Units'] = df[col].iloc[0]
    time_series_metadata.ix[col, 'Source'] = 'Farmer_Lafond'

df = df.iloc[2:]

for col in df.columns:
    time_series_metadata.ix[col, 'n'] = df[col].notnull().sum()
    time_series_metadata.ix[col, 'Start'] = df[col].dropna().index[0]
    time_series_metadata.ix[col, 'Stop'] = df[col].dropna().index[-1]    

In [4]:
time_series = df.copy()

In [6]:
from os import listdir
from numpy import sort
files = sort(listdir(data_directory+'original/Benson_Magee_Data/'))

In [7]:
for f in files[::-1]:
    if f.endswith('.xlsx') and 'combo' not in f:
        df = pd.read_excel(data_directory+'original/Benson_Magee_Data/'+f,sheetname='rawdata',index_col=0)
        col = f.split('_v1')[0]
        time_series_metadata.ix[col, 'Units'] = df.columns[0]
        time_series_metadata.ix[col, 'Source'] = 'Benson_Magee'
        time_series_metadata.ix[col, 'n'] = df.dropna().shape[0]
        time_series_metadata.ix[col, 'Start'] = df.dropna().index[0]
        time_series_metadata.ix[col, 'Stop'] = df.dropna().index[-1]
        df.rename(columns={df.columns[0]:col}, inplace=True)
        df.index = df.index.astype('float')
        df = 1/df
        df = df.groupby(level=0).min()
        time_series = time_series.join(df, how='outer')

In [8]:
time_series = time_series.astype('float')

In [ ]:
time_series.to_csv(data_directory+'time_series.csv')
time_series_metadata.to_csv(data_directory+'time_series_metadata.csv')

In [ ]:
%pylab inline

for c in time_series.columns:
    figure()
    time_series[c].dropna().plot(legend=False)
    yscale('log')
    title(c)

`%matplotlib` prevents importing * from pylab and numpy
/home/jeffrey_alstott/anaconda3/lib/python3.4/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [ ]:
time_series_metadata[time_series_metadata['Source']=='Farmer_Lafond'].shape

In [ ]:
time_series_metadata[time_series_metadata['Source']=='Benson_Magee'].shape